In [ ]:
import pandas as pd
import re

# Sample data
df = pd.DataFrame({
    'raw_date': ['Mon, 28th May 2020 12:34:56 GMT', 'Tue, 1st Jan 2019 05:00:00 GMT']
})

# Remove day suffixes like 'st', 'nd', 'rd', 'th' using regex
df['cleaned_date'] = df['raw_date'].apply(lambda x: re.sub(r'(\d+)(st|nd|rd|th)', r'\1', x))

# Parse into datetime format
df['parsed_date'] = pd.to_datetime(df['cleaned_date'], format='%a, %d %b %Y %H:%M:%S %Z', errors='coerce')

# Optional: just keep the date part (no time)
df['just_date'] = df['parsed_date'].dt.date

print(df[['raw_date', 'just_date']])


In [ ]:
import matplotlib.pyplot as plt

# Sample data
years = ['2010', '2025']
car1_prices = [15000, 18000]
car2_prices = [20000, 25000]
car3_prices = [17000, 22000]

# Plot
fig, ax = plt.subplots()

# Stacked bars
bar1 = ax.bar(years, car1_prices, label='Car 1')
bar2 = ax.bar(years, car2_prices, bottom=car1_prices, label='Car 2')

# Compute bottoms for car 3 (stacked on car1 + car2)
car1_plus_car2 = [c1 + c2 for c1, c2 in zip(car1_prices, car2_prices)]
bar3 = ax.bar(years, car3_prices, bottom=car1_plus_car2, label='Car 3')

# Add labels and legend
ax.set_ylabel('Total Price ($)')
ax.set_title('Stacked Car Prices in 2010 vs 2025')
ax.legend()

plt.tight_layout()
plt.show()


In [ ]:
import shap
import seaborn as sns
import matplotlib.pyplot as plt

# Get SHAP interaction values
explainer = shap.TreeExplainer(model)
shap_interaction_values = explainer.shap_interaction_values(X)  # shape: [n_samples, n_features, n_features]

# Define feature of interest
feature_name = 'A'
feature_index = list(X.columns).index(feature_name)

# Interaction values of 'A' with every other feature (averaged across samples)
interaction_strengths = np.abs(shap_interaction_values[:, feature_index, :]).mean(axis=0)

# Create DataFrame for plotting
interaction_df = pd.DataFrame({
    'feature': X.columns,
    'interaction_strength': interaction_strengths
}).sort_values(by='interaction_strength', ascending=False)

# Optional: Drop self-interaction if you only want cross-feature effects
interaction_df = interaction_df[interaction_df.feature != feature_name]

# Plot
plt.figure(figsize=(10, 6))
sns.barplot(data=interaction_df, x='interaction_strength', y='feature')
plt.title(f"Mean SHAP Interaction Strengths with '{feature_name}'")
plt.xlabel("Mean |SHAP Interaction Value|")
plt.ylabel("Interacting Feature")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
CMAKE_ARGS="-DLLAMA_CUDA=OFF -DLLAMA_METAL=OFF -DLLAMA_CLBLAST=OFF -DLLAMA_BLAS=OFF" \
pip install --force-reinstall --no-cache-dir --no-binary llama-cpp-python llama-cpp-python

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import zscore

# --- Step 1: Simulate weekly purchase data ---
np.random.seed(0)
weeks = pd.date_range("2023-01-01", periods=52, freq='W')
states = ['Texas', 'Oklahoma', 'Louisiana', 'California', 'New York']

def make_trend(base, noise=5):
    return base + np.random.normal(0, noise, size=len(weeks))

data = {
    'Texas': make_trend(np.linspace(100, 200, 52)),
    'Oklahoma': make_trend(np.linspace(90, 180, 52)),
    'Louisiana': make_trend(np.linspace(105, 205, 52)),
    'California': make_trend(np.linspace(300, 400, 52)),
    'New York': make_trend(np.linspace(200, 100, 52)),
}

df = pd.DataFrame(data, index=weeks)

# --- Step 2: Normalize trends (Z-score) ---
df_normalized = df.apply(zscore)

# --- Step 3: Hyper-tune K using silhouette score ---
X = df_normalized.T  # states as rows
sil_scores = {}
for k in range(2, min(len(X), 10)):
    model = KMeans(n_clusters=k, random_state=0)
    labels = model.fit_predict(X)
    score = silhouette_score(X, labels)
    sil_scores[k] = score

best_k = max(sil_scores, key=sil_scores.get)
print(f"Best number of clusters: {best_k} with silhouette score: {sil_scores[best_k]:.3f}")

# --- Step 4: Fit final model and visualize ---
final_model = KMeans(n_clusters=best_k, random_state=0)
labels = final_model.fit_predict(X)
cluster_df = pd.DataFrame({'state': X.index, 'cluster': labels})
print(cluster_df)

# --- Plot silhouette scores ---
plt.figure(figsize=(8, 5))
plt.plot(list(sil_scores.keys()), list(sil_scores.values()), marker='o')
plt.xlabel("Number of Clusters (k)")
plt.ylabel("Silhouette Score")
plt.title("Silhouette Score vs. Number of Clusters")
plt.grid(True)
plt.xticks(range(2, min(len(X), 10)))
plt.axvline(x=best_k, linestyle='--', color='red', label=f'Best k = {best_k}')
plt.legend()
plt.tight_layout()
plt.show()


# --- Step 5: Plot each cluster's normalized time trends ---
for cluster in cluster_df['cluster'].unique():
    cluster_states = cluster_df[cluster_df['cluster'] == cluster]['state']
    df_normalized[cluster_states].plot(title=f"Cluster {cluster} Trends")
    plt.ylabel("Z-Score Normalized Purchases")
    plt.show()

# --- Step 6: Correlation heatmap ---
corr = df.corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.title("Raw Correlation of Purchase Trends")
plt.show()

# --- Step 7: Intra-cluster correlation check ---
def mean_cluster_corr(corr_matrix, cluster_df):
    results = {}
    for cluster in cluster_df['cluster'].unique():
        members = cluster_df[cluster_df['cluster'] == cluster]['state']
        intra = corr_matrix.loc[members, members].values
        intra_corr = intra[np.triu_indices_from(intra, k=1)]
        results[f"Cluster {cluster} mean corr"] = np.mean(intra_corr)
    return results

print(mean_cluster_corr(corr, cluster_df))


In [ ]:
# Convert dates to datetime if they aren't already
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date']   = pd.to_datetime(df['end_date'])

# 1️⃣ Recode: 1 if end_date is after start_date, else 0
df['date_after_flag'] = (df['end_date'] > df['start_date']).astype(int)

# 2️⃣ Recode: 1 if 'comments' contains 'hi' (case-insensitive), else 0
df['contains_hi_flag'] = df['comments'].str.contains('hi', case=False, na=False).astype(int)

# Sort so earliest date comes first
df = df.sort_values(by=['email', 'date'])

# Drop duplicates, keeping first occurrence (earliest date)
df_first_date = df.drop_duplicates(subset='email', keep='first')

# 1️⃣ Boolean masks
mask_saiid = df['campaign'].str.contains('saiid', case=False, na=False)
mask_not_saiid = ~mask_saiid

# 2️⃣ Sets of emails
emails_saiid = set(df.loc[mask_saiid, 'email'])
emails_not_saiid = set(df.loc[mask_not_saiid, 'email'])

# 3️⃣ Intersection and differences
emails_both = emails_saiid & emails_not_saiid
emails_only_saiid = emails_saiid - emails_both
emails_only_not_saiid = emails_not_saiid - emails_both

# 4️⃣ Counts
print("Count in both:", len(emails_both))
print("Count only in saiid:", len(emails_only_saiid))
print("Count only in non-saiid:", len(emails_only_not_saiid))

# Conditional logic
df['final_flag'] = (
    ((df['one_column'] == 1) & (df['min_date'] > df['application_date'])) |
    ((df['one_column'] == 0) & (df['application_date'] > fixed_date))
).astype(int)

In [ ]:
campaign_substr = 'hi'  # the substring to search for

# 1) Which rows are in a matching campaign?
mask = df['campaign'].astype(str).str.contains(campaign_substr, case=False, na=False)

# 2) All emails that EVER had a matching campaign
emails_with_match = set(df.loc[mask, 'email'])

# 3) Flag every row for those emails (1 if the email ever used the campaign, else 0)
df['ever_hi'] = df['email'].isin(emails_with_match).astype(int)

# 4) First date each email was associated with a matching campaign
first_hi_per_email = (
    df.loc[mask]
      .groupby('email', as_index=False)['date']
      .min()
      .set_index('email')['date']
)

# Map to every row; emails without a match get NaT
df['first_hi_date'] = df['email'].map(first_hi_per_email)

# (optional) If you'd rather fill non-matching emails with a sentinel string:
# df['first_hi_date'] = df['first_hi_date'].dt.strftime('%Y-%m-%d').fillna('never')

In [ ]:
# Make sale column human-readable
df["sale_label"] = df["sale"].map({0: "Non-Sale", 1: "Sale"})

# Set style
sns.set(style="whitegrid", palette="Set2")

# Boxplot
plt.figure(figsize=(6, 5))
ax = sns.boxplot(x="sale_label", y="score", data=df, width=0.5)

# Overlay stripplot to show individual points
sns.stripplot(x="sale_label", y="score", data=df, 
              color="black", size=4, alpha=0.6, jitter=True)

# Titles and labels
ax.set_title("Model Score Distribution by Outcome", fontsize=14, weight="bold")
ax.set_xlabel("")
ax.set_ylabel("Model Score (1–5)", fontsize=12)

plt.tight_layout()
plt.show()